In [1]:
import numpy as np

# 1. 建立詞彙表
corpus = ["I like to play soccer", "Soccer is a fun sport", "I enjoy playing soccer"]
words = ' '.join(corpus).split()
vocab = list(set(words))

# 2. 初始化共現矩陣
co_occurrence_matrix = np.zeros((len(vocab), len(vocab)))

# 3. 定義前後窗口
window_size = 2

# 4. 計算數值並更新共現矩陣
for sentence in corpus:
    tokens = sentence.split()
    for i in range(len(tokens)):
        for j in range(max(0, i - window_size), min(len(tokens), i + window_size + 1)):
            if i != j:
                word_i, word_j = tokens[i], tokens[j]
                if word_i in vocab and word_j in vocab:
                    index_i, index_j = vocab.index(word_i), vocab.index(word_j)
                    co_occurrence_matrix[index_i][index_j] += 1

print(co_occurrence_matrix)

[[0. 0. 1. 1. 0. 0. 0. 0. 1. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 1. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 1. 0. 1. 0.]
 [1. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 1. 0. 1. 0. 1. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 1. 1.]
 [0. 1. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0.]
 [1. 0. 1. 0. 0. 0. 1. 0. 0. 0. 1. 0.]
 [0. 1. 0. 0. 1. 1. 0. 1. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 1. 0. 1. 0. 0. 0.]
 [1. 0. 0. 1. 0. 0. 1. 0. 0. 0. 0. 0.]]


In [2]:
import torch.nn as nn
import torch
class GloVe(nn.Module):
    def __init__(self, vocab_size, embedding_dim):
        super().__init__()
        self.embeddings = nn.Embedding(vocab_size, embedding_dim)
        self.bias_target = nn.Embedding(vocab_size, 1)
        self.bias_context = nn.Embedding(vocab_size, 1)

    def forward(self, target, context):
        embed_target = self.embeddings(target)
        embed_context = self.embeddings(context)
        bias_target = self.bias_target(target).squeeze()
        bias_context = self.bias_context(context).squeeze()
        
        dot_product = torch.sum(embed_target * embed_context, dim=1)
        log_co_occurrences = torch.log(co_occurrence_matrix[target, context]) 
        loss = torch.mean((dot_product + bias_target + bias_context - log_co_occurrences))
        return loss
        

co_occurrence_matrix = torch.LongTensor(co_occurrence_matrix)

vocab_size = co_occurrence_matrix.shape[0]
embedding_dim = 768

model = GloVe(vocab_size, embedding_dim)
target, context = torch.where(co_occurrence_matrix > 0)

model(target, context)

tensor(5.6144, grad_fn=<MeanBackward0>)

In [ ]:
import torch
import torch.nn as nn

# 创建一个HierarchicalSoftmax模型实例，假设输入大小为input_size，类别数为num_classes
input_size = 100
num_classes = 10
model = HierarchicalSoftmax(input_size, num_classes)

# 生成假造的输入数据，假设批量大小为batch_size
batch_size = 32
input_data = torch.randn(batch_size, input_size)

# 将输入数据传递给模型的forward方法来获取输出
outputs = model(input_data)